In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

this an example on how to do FCI fro the 4 index integrals, or the 2 electron integrals. 

In [2]:
def Add_Block_Matrices(Paa, Pbb):
    Pab = np.zeros(Paa.shape)
    Pba = np.zeros(Paa.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# basic setup of a molecule
element = "He"
mol = gto.Mole()
mol.atom = f'''{element}  0 0 0'''
# mol.spin = 1
# mol.charge = -1
mol.basis = '631-g'
mol.build()
eri_ao = mol.intor('int2e')

In [4]:
# run Hartree Fock
mf = scf.HF(mol)
mf.kernel()

converged SCF energy = -2.85516042615445


np.float64(-2.8551604261544465)

In [5]:
# Harvest HF quantities
M = mol.nao
h1_ao = mf.get_hcore()
C = mf.mo_coeff
h1_mo = C.T@h1_ao@C
h2_mo = ao2mo.kernel(mol, C)


In [6]:
# for reference the simplest way
cisolver = fci.FCI(mol, mf.mo_coeff)
e_1, fcivec = cisolver.kernel()

In [7]:
# and this is the purpose of this script 
e_2, ci = fci.direct_spin0.kernel(h1_mo, h2_mo, M, mol.nelec, ecore=mf.energy_nuc())

In [8]:
eri_mo = np.asarray(ao2mo.restore(1, h2_mo, M))
e_3, ci = fci.direct_spin0.kernel(h1_mo, eri_mo, M, mol.nelec, ecore=mf.energy_nuc())

In [9]:
e_4, ci = fci.direct_spin0.kernel(h1_ao, eri_ao,M, mol.nelec, ecore=mf.energy_nuc())

In [10]:
print(e_1, e_2, e_3, e_4)

-2.8701621389008225 -2.8701621389008234 -2.8701621389008234 -3.661036580713073


In [11]:
print(eri_ao.shape, eri_mo.shape, h2_mo.shape, h1_mo.shape[0])

(2, 2, 2, 2) (2, 2, 2, 2) (3, 3) 2


In [12]:
print(eri_ao)

[[[[1.54393568 0.78544719]
   [0.78544719 0.80395186]]

  [[0.78544719 0.42702546]
   [0.42702546 0.47618957]]]


 [[[0.78544719 0.42702546]
   [0.42702546 0.47618957]]

  [[0.80395186 0.47618957]
   [0.47618957 0.61593794]]]]


In [13]:
print(eri_mo)

[[[[ 1.02690717 -0.31649047]
   [-0.31649047  0.85813333]]

  [[-0.31649047  0.2276705 ]
   [ 0.2276705  -0.25555355]]]


 [[[-0.31649047  0.2276705 ]
   [ 0.2276705  -0.25555355]]

  [[ 0.85813333 -0.25555355]
   [-0.25555355  0.7663629 ]]]]


In [14]:
ao_mo_eri = eri_ao
for i in range(4):
   ao_mo_eri = np.tensordot(ao_mo_eri, C, axes=1).transpose(3, 0, 1, 2)

In [15]:
print(ao_mo_eri)

[[[[ 1.02690717 -0.31649047]
   [-0.31649047  0.85813333]]

  [[-0.31649047  0.2276705 ]
   [ 0.2276705  -0.25555355]]]


 [[[-0.31649047  0.2276705 ]
   [ 0.2276705  -0.25555355]]

  [[ 0.85813333 -0.25555355]
   [-0.25555355  0.7663629 ]]]]
